In [7]:
import pandas as pd
import plotly.graph_objs as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import webbrowser
import threading
from dash.dash_table import DataTable  # 올바른 dash_table 가져오기

# Dash 앱 실행 함수
def run_dash():
    app.run_server(debug=True)  # debug=True로 설정하여 자동 리로드 활성화

# 엑셀 파일 불러오기
file_path = r'C:\Users\jh\Desktop\쇼핑몰-매출분석데이터.xlsx'
df = pd.read_excel(file_path, sheet_name='기록데이터')

# 데이터 변환: '2024-01', '2024-02'와 같은 열들을 '시간' 열과 '값' 열로 변환
df_long = df.melt(id_vars=['판매자관리코드', '쇼핑몰', '오픈마켓'], 
                  value_vars=df.columns[3:],  # 3번째 열부터 월별 데이터
                  var_name='시간', value_name='값')

# '시간' 열을 datetime 형식으로 변환
df_long['시간'] = pd.to_datetime(df_long['시간'], errors='coerce', format='%Y-%m')
df_long['시간'] = df_long['시간'].dt.strftime('%Y-%m')

# 데이터 전처리 (중복 제거)
df_unique = df_long.drop_duplicates(subset=['판매자관리코드', '쇼핑몰', '오픈마켓', '시간'])

# 각 카테고리별 고유 값 추출
판매자관리코드_options = [{'label': str(code), 'value': code} for code in df['판매자관리코드'].unique()]
쇼핑몰_options = [{'label': str(shop), 'value': shop} for shop in df['쇼핑몰'].unique()]
오픈마켓_options = [{'label': str(market), 'value': market} for market in df['오픈마켓'].unique()]
전체_option = {'label': '전체', 'value': '전체'}

# Dash 앱 초기화
app = dash.Dash(__name__)

# 그래프를 그리기 위한 함수
def generate_graph(selected_판매자관리코드, selected_쇼핑몰, selected_오픈마켓):
    filtered_data = df_unique

    if selected_판매자관리코드 != '전체':
        filtered_data = filtered_data[filtered_data['판매자관리코드'] == selected_판매자관리코드]
    if selected_쇼핑몰 != '전체':
        filtered_data = filtered_data[filtered_data['쇼핑몰'] == selected_쇼핑몰]
    if selected_오픈마켓 != '전체':
        filtered_data = filtered_data[filtered_data['오픈마켓'] == selected_오픈마켓]

    if selected_판매자관리코드 == '전체' or selected_쇼핑몰 == '전체' or selected_오픈마켓 == '전체':
        filtered_data = filtered_data.groupby('시간', as_index=False)['값'].sum()

    title = f"{selected_판매자관리코드 if selected_판매자관리코드 != '전체' else '전체'} - " \
            f"{selected_쇼핑몰 if selected_쇼핑몰 != '전체' else '전체'} - " \
            f"{selected_오픈마켓 if selected_오픈마켓 != '전체' else '전체'} 월별 그래프"

    if selected_판매자관리코드 == '전체' or selected_쇼핑몰 == '전체' or selected_오픈마켓 == '전체':
        fig = px.line(filtered_data, x='시간', y='값', title=title)
    else:
        fig = px.line(filtered_data, x='시간', y='값', color='판매자관리코드', title=title)

    # 선 색을 빨간색으로, 텍스트 추가
    fig.update_traces(
        line=dict(color='red'),   # 빨간색 선
        text=filtered_data['값'],  # 숫자 표시를 위한 텍스트 추가
        textposition="top center"  # 텍스트 위치 설정
    )

    return fig

def generate_table(selected_판매자관리코드, selected_쇼핑몰, selected_오픈마켓, sort_by_column):
    filtered_data = df_unique

    # 필터링: 각 선택 사항에 맞게 필터링
    if selected_판매자관리코드 != '전체':
        filtered_data = filtered_data[filtered_data['판매자관리코드'] == selected_판매자관리코드]
    if selected_쇼핑몰 != '전체':
        filtered_data = filtered_data[filtered_data['쇼핑몰'] == selected_쇼핑몰]
    if selected_오픈마켓 != '전체':
        filtered_data = filtered_data[filtered_data['오픈마켓'] == selected_오픈마켓]

    # '값' 열을 숫자형으로 변환 (NaN 값 처리)
    filtered_data['값'] = pd.to_numeric(filtered_data['값'], errors='coerce')

    # NaN 값이 포함되지 않도록 필터링
    filtered_data = filtered_data.dropna(subset=['값'])

    # '판매자관리코드'와 '시간'별로 그룹화하여 합계값 계산
    monthly_sum = filtered_data.groupby(['판매자관리코드', '시간'])['값'].sum().reset_index()

    # 각 판매자관리코드별로 월별 합계 중 최대값 계산
    max_value_per_seller = monthly_sum.groupby('판매자관리코드')['값'].max().reset_index()
    max_value_per_seller.columns = ['판매자관리코드', '최대값']

    # 각 판매자관리코드별로 최소값, 평균값, 합계값 계산
    min_value_per_seller = monthly_sum.groupby('판매자관리코드')['값'].min().reset_index()
    mean_value_per_seller = monthly_sum.groupby('판매자관리코드')['값'].mean().reset_index()
    sum_value_per_seller = monthly_sum.groupby('판매자관리코드')['값'].sum().reset_index()

    # 최종 테이블 결합
    table_data_summary = pd.merge(max_value_per_seller, min_value_per_seller, on='판매자관리코드')
    table_data_summary = pd.merge(table_data_summary, mean_value_per_seller, on='판매자관리코드')
    table_data_summary = pd.merge(table_data_summary, sum_value_per_seller, on='판매자관리코드')

    # 3개월 평균값과 6개월 평균값 계산
    table_data_summary['3개월 평균값'] = table_data_summary['판매자관리코드'].apply(
        lambda x: calculate_period_average(x, 3, monthly_sum))
    table_data_summary['6개월 평균값'] = table_data_summary['판매자관리코드'].apply(
        lambda x: calculate_period_average(x, 6, monthly_sum))

    # 컬럼 이름 변경
    table_data_summary.columns = ['판매자관리코드', '최대값', '최소값', '평균값', '합계값', '3개월 평균값', '6개월 평균값']

    # NaN 처리: NaN 값을 0으로 대체
    table_data_summary['최대값'] = table_data_summary['최대값'].fillna(0).round(1)
    table_data_summary['최소값'] = table_data_summary['최소값'].fillna(0).round(1)
    table_data_summary['평균값'] = table_data_summary['평균값'].fillna(0).round(1)
    table_data_summary['합계값'] = table_data_summary['합계값'].fillna(0).round(1)
    table_data_summary['3개월 평균값'] = table_data_summary['3개월 평균값'].fillna(0).round(1)
    table_data_summary['6개월 평균값'] = table_data_summary['6개월 평균값'].fillna(0).round(1)

    # 정렬
    if sort_by_column:
        table_data_summary = table_data_summary.sort_values(by=sort_by_column, ascending=True)

    return table_data_summary.to_dict('records')

# 최근 N개월 평균값 계산 함수
def calculate_period_average(seller_code, months, monthly_sum):
    # 판매자관리코드에 해당하는 데이터만 필터링
    seller_data = monthly_sum[monthly_sum['판매자관리코드'] == seller_code]

    # 최신 월(가장 큰 '시간')을 구함
    latest_month = seller_data['시간'].max()

    # 최신 월을 기준으로 N개월을 찾기 위한 기간 설정
    period_end = pd.to_datetime(latest_month)
    period_start = period_end - pd.DateOffset(months=months)

    # 기간에 해당하는 데이터만 필터링
    period_data = seller_data[(pd.to_datetime(seller_data['시간']) >= period_start) & 
                              (pd.to_datetime(seller_data['시간']) <= period_end)]

    # N개월 기간 동안의 평균값 계산
    period_avg = period_data['값'].mean()

    return period_avg

# Dash 앱 레이아웃 설정
app.layout = html.Div([
    html.H1("월별 판매 분석"),
    dcc.Dropdown(
        id='판매자관리코드-selector',
        options=[전체_option] + 판매자관리코드_options,
        value='전체',
        placeholder='판매자관리코드 선택',
        clearable=True
    ),
    dcc.Dropdown(
        id='쇼핑몰-selector',
        options=[전체_option] + 쇼핑몰_options,
        value='전체',
        placeholder='쇼핑몰 선택',
        clearable=True
    ),
    dcc.Dropdown(
        id='오픈마켓-selector',
        options=[전체_option] + 오픈마켓_options,
        value='전체',
        placeholder='오픈마켓 선택',
        clearable=True
    ),
    dcc.Graph(id='category-graph'),  # 그래프에서 버튼을 제외
    DataTable(  # 수정된 부분
        id='summary-table',
        columns=[
            {'name': '판매자관리코드', 'id': '판매자관리코드'},
            {'name': '최대값', 'id': '최대값'},
            {'name': '최소값', 'id': '최소값'},
            {'name': '평균값', 'id': '평균값'},
            {'name': '합계값', 'id': '합계값'},
            {'name': '3개월 평균값', 'id': '3개월 평균값'},
            {'name': '6개월 평균값', 'id': '6개월 평균값'},
        ],
        style_table={'display': 'flex', 'justify-content': 'center', 'align-items': 'center', 'width': '100%', 'overflowY': 'auto'},
        style_cell={'textAlign': 'center', 'minWidth': '100px', 'width': '100px', 'maxWidth': '100px'},
        sort_action='native',  # 기본적으로 정렬 기능 활성화
        sort_mode='single',    # 단일 열 정렬
        sort_by=[{'column_id': '최대값', 'direction': 'desc'}],  # 초기 정렬 기준
        style_data_conditional=[
            {
                'if': {
                    'filter_query': '{3개월 평균값} < {6개월 평균값}',  # 3개월 평균값이 6개월 평균값보다 작은 조건
                    'column_id': '3개월 평균값'
                },
                'backgroundColor': '#f8d7da',  # 옅은 빨간색 배경
                'color': 'black'  # 텍스트 색상
            },
            {
                'if': {
                    'filter_query': '{3개월 평균값} > {6개월 평균값}',  # 3개월 평균값이 6개월 평균값보다 작은 조건
                    'column_id': '3개월 평균값'
                },
                'backgroundColor': '#d4edda',  # 옅은 빨간색 배경
                'color': 'black'  # 텍스트 색상
            }
        ]
    ),
    dcc.Dropdown(  # 테이블용 버튼
        id='정렬옵션',
        options=[  # 테이블 정렬 옵션
            {'label': '최대값 기준', 'value': '최대값'},
            {'label': '최소값 기준', 'value': '최소값'},
            {'label': '평균값 기준', 'value': '평균값'},
            {'label': '합계값 기준', 'value': '합계값'},
        ],
        value='최대값',
        placeholder='정렬 옵션 선택'
    ),
])

# 콜백 설정
@app.callback(
    Output('category-graph', 'figure'),
    Output('summary-table', 'data'),
    Input('판매자관리코드-selector', 'value'),
    Input('쇼핑몰-selector', 'value'),
    Input('오픈마켓-selector', 'value'),
    Input('정렬옵션', 'value')
)
def update_output(selected_판매자관리코드, selected_쇼핑몰, selected_오픈마켓, sort_by_column):
    graph = generate_graph(selected_판매자관리코드, selected_쇼핑몰, selected_오픈마켓)
    table_data = generate_table(selected_판매자관리코드, selected_쇼핑몰, selected_오픈마켓, sort_by_column)
    return graph, table_data

# Dash 서버 실행
if __name__ == '__main__':
    thread = threading.Thread(target=run_dash, daemon=True)
    thread.start()
    webbrowser.open_new("http://127.0.0.1:8050/")


test용



In [5]:
import git
import os

# 작업 디렉터리 설정
repo_dir = r'C:\Users\jh\크롤링'

# 로컬 Git 리포지토리 열기 (리포지토리가 없다면 init을 통해 초기화)
repo = git.Repo.init(repo_dir)

# GitHub 저장소와 연결 (HTTPS 인증으로 변경)
remote_url = 'https://github.com/koolca1219/T1.git'

# 이미 원격 저장소가 설정되어 있다면, 이를 업데이트
if 'origin' not in [remote.name for remote in repo.remotes]:
    repo.create_remote('origin', remote_url)
else:
    origin = repo.remotes.origin
    origin.set_url(remote_url)

# 파일 추가 및 커밋
file_path = os.path.join(repo_dir, '그래프분석V3.ipynb')

# 커밋 메시지
commit_message = "Add 그래프분석V3.ipynb - 버전 1"

# 파일을 Git에 추가하고 커밋
repo.index.add([file_path])
repo.index.commit(commit_message)

# GitHub에 푸시
origin.push('main')
